# MFT Asynchronous QC

In [1]:
_period = "LHC22o"
_run = 527863
_pass1 = "apass1"
_pass2 = "apass2"

In [2]:
import ROOT
from ROOT import gStyle
%jsroot on

print(f"period: {_period}")
print(f"run:    {_run}")
print(f"pass1:  {_pass1}")
print(f"pass2:  {_pass2}")

# global variables:
f_p1 = ROOT.TFile.Open(f"Results/{_period}/runsRootFiles/{_run}_{_pass1}.root")
f_p2 = ROOT.TFile.Open(f"Results/{_period}/runsRootFiles/{_run}_{_pass2}.root")

names = [
# track position (6 in total)
"tracks_mMFTTrackEtaPhi_5_MinClusters",
"tracks_mMFTTrackXY_5_MinClusters",
"tracks_mMFTTrackEtaPhi_7_MinClusters",
"tracks_mMFTTrackXY_7_MinClusters",
"tracks_mMFTTrackEtaPhi_8_MinClusters",
"tracks_mMFTTrackXY_8_MinClusters",
# tracks (11 in total)
"tracks_mMFTTrackEta",
"tracks_mMFTTrackNumberOfClusters",
"tracks_mMFTTrackPhi",
"tracks_mMFTTrackTanl",
"tracks_mNOfTracksTime",
"tracks_mMFTTrackInvQPt",
"tracks_CA_mMFTCATrackPt",
"tracks_LTF_mMFTLTFTrackPt",
"tracks_CA_mMFTCATrackEta",
"tracks_LTF_mMFTLTFTrackEta",
"tracks_mMFTTracksBC",
# clusters (4 in total)
"clusters_mMFTClusterPatternIndex",
"clusters_mMFTClusterSensorIndex",
"clusters_mMFTClustersROFSize",
"clusters_mNOfClustersTime"]

# which histograms to plot with log scale on y-axis
logScales = [
"tracks_CA_mMFTCATrackPt",
"tracks_LTF_mMFTLTFTrackPt",
"clusters_mMFTClusterPatternIndex",
"clusters_mMFTClusterSensorIndex",
"clusters_mMFTClustersROFSize"]

gStyle.SetTitleX(0.3)
gStyle.SetTitleY(1.0)
gStyle.SetOptStat(0)

legArr1D = ROOT.TObjArray(len(names))
legArr1D.SetOwner()
i = 0
for histoName in names:
    l = ROOT.TLegend(0.60,0.90,0.95,1.00)
    l.SetBorderSize(0)
    l.SetFillColor(0)
    l.SetFillStyle(0)
    l.SetTextFont(42)
    legArr1D.AddAt(l,i)
    i+=1
    
def plotComparison1D(canvasName, drawOption, nCols, nRows, histoNames, legFirstIdx):
    c = ROOT.TCanvas(canvasName,"",nCols*400,nRows*300)
    c.Divide(nCols,nRows)
    i = 1
    for histoName in histoNames:
        h_p1 = f_p1.Get(histoName)
        h_p2 = f_p2.Get(histoName)
        norm1 = 0 
        norm2 = 0 
        max1 = 0
        max2 = 0
        if h_p1 != None:
            norm1 = h_p1.Integral()
            h_p1.Scale(1./norm1)
            max1 = h_p1.GetMaximum()
        if h_p2 != None:
            norm2 = h_p2.Integral()
            h_p2.Scale(1./norm2)
            max2 = h_p2.GetMaximum()
        maxGlob = max1
        if max2 > max1:
            maxGlob = max2
        c.cd(i)
        logScale = False
        for name in logScales:
            if histoName == name:
                logScale = True
        if logScale == True:
            c.cd(i).SetLogy()
        l = legArr1D.At(legFirstIdx)
        if h_p1 != None:
            l.AddEntry(h_p1,f"{_pass1}: {norm1:.3e}")
            h_p1.SetLineColor(4)
            if logScale == False:
                h_p1.GetYaxis().SetRangeUser(0.,maxGlob*1.05)
            h_p1.Draw(f"{drawOption}")
        if h_p2 != None: 
            l.AddEntry(h_p2,f"{_pass2}: {norm2:.3e}")
            h_p2.SetLineColor(2)
            h_p2.Draw(f"{drawOption} same")
        l.Draw()
        i+=1
        legFirstIdx+=1
    return c

Welcome to JupyROOT 6.26/10
period: LHC22o
run:    527863
pass1:  apass1
pass2:  apass2


## TRACK POSITION:

In [3]:
nCols = 2
nRows = 6
c = ROOT.TCanvas("trackPos","",nCols*400,nRows*300)
c.Divide(nCols,nRows)
empty = ROOT.TObject()

legArr2D = ROOT.TObjArray(12)
legArr2D.SetOwner()
i = 0
for histoName in names[0:12]:
    l = ROOT.TLegend(0.6,0.93,0.95,1.00)
    l.SetBorderSize(0)
    l.SetFillColor(0)
    l.SetFillStyle(0)
    l.SetTextFont(42)
    legArr2D.AddAt(l,i)
    i+=1

i = 1
for histoName in names[0:6]:
    h_p1 = f_p1.Get(histoName)
    h_p2 = f_p2.Get(histoName)
    c.cd(2*i-1)
    if h_p1 != None:
        h_p1.Draw("colz")
    l_p1 = legArr2D.At(2*i-2)
    l_p1.AddEntry(empty,f"{_pass1}: {h_p1.Integral():.3e}","")
    l_p1.Draw()
    c.cd(2*i)
    if h_p2 != None:
        h_p2.Draw("colz")
    l_p2 = legArr2D.At(2*i-1)
    l_p2.AddEntry(empty,f"{_pass2}: {h_p2.Integral():.3e}","")
    l_p2.Draw()
    i+=1
c.Draw()

## TRACK PROPERTIES:

In [4]:
c2 = plotComparison1D("trackProp","hist",2,6,names[6:17],6)
c2.Draw()

## CLUSTER PROPERTIES:

In [5]:
c3 = plotComparison1D("clusterProp","hist",2,2,names[17:21],17)
c3.Draw()